In [1]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [2]:
import data_utils
import embedding_utils
import display_utils

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
np.random.seed(1001)
tf.set_random_seed(1001)

### Load data

In [5]:
VOCAB_SIZE  = 50000
#load dataset
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [6]:

#load conter-fitted word 
dist_mat = np.load('aux_files/dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)


In [7]:
doc_len = [len(dataset.val_X[i]) for i in 
           range(len(dataset.val_X))]

### Demonstrating how we find the most similar words

In [8]:
src_word = dataset.dict['好']
src_word
dataset.inv_full_dict[50000]

'伟'

In [9]:
for i in range(6230, 6232):
    src_word = i
    nearest, nearest_dist = embedding_utils.pick_most_similar_words(src_word, dist_mat,20, 0.5)
        
    print('Closest to `%s` are:' %(dataset.inv_dict[src_word]))
    for w_id, w_dist in zip(nearest, nearest_dist):
          print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

    print('----')

Closest to `国税局` are:
----
Closest to `翼` are:
 --  机翼   0.22788355864099952
 --  辅   0.22916311992500127
 --  翅膀   0.2738878111520009
 --  援   0.3341176668279999
 --  辅助   0.34568997297100035
 --  副   0.3563555582999989
 --  扶持   0.36099929984099965
 --  救援   0.3625574274400005
 --  支架   0.3828246505230002
 --  帮   0.390361236298
 --  援助   0.3941633762969998
 --  养   0.4035929243710008
 --  救   0.4049656322590014
 --  架   0.417898789983
 --  赞助   0.42399925209200195
 --  扶   0.42721877353100135
 --  资助   0.43061200651000053
 --  助益   0.4325585738910007
 --  声援   0.43391956708499957
 --  托   0.43443959343899996
----


### Loading the NLI model

In [10]:
from NLImodel.models import NLImodelClass


In [11]:
model = NLImodelClass(is_train=False)


Learning Rate:  0.0007
Model Loaded.


In [12]:
dataset.val_X[0], dataset.val_Y[0], dataset.val_Z[0], np.argmax(dataset.val_Z[0])

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
          328, 50000,    29,  2923,  1991,   236,   108, 50000,  2631,
           29,   106,  2348, 50000,  1099,    29,  2106,    15,    52,
          620,    62, 50000], dtype=int32),
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    29,   900,   262,    15,    52,
         3915,  1670, 50000], dtype=int32),
 array([0., 0., 1.]),
 2)

In [13]:
np.argmax(model.predict(dataset.val_X[0], dataset.val_X[0])) 

1

## Try Attack

In [14]:
from attacks import GreedyAttack, GeneticAtack

### Main Attack 

In [15]:
pop_size = 32
n1 = 8
goog_lm = None

ga_atttack = GeneticAtack(model, model, model, dataset, dist_mat, 
                                  skip_list,
                                  goog_lm, max_iters=30, 
                                   pop_size=pop_size,
                                  
                                  n1 = n1,
                                  n2 = 4,
                                 use_lm = False, use_suffix=False)

In [16]:
len(dataset.val_Z)

5010

In [31]:
TEST_SIZE = len(dataset.val_Z)#1000
#use val data for attack
#test_idx = np.random.choice(len(dataset.val_Z), TEST_SIZE, replace=False)#随机index,总共1000个
test_idx = [x for x in range(len(dataset.val_Z))]
test_len = []
for i in range(TEST_SIZE):
    test_len.append(len(dataset.val_Y[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_list_no_max = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(TEST_SIZE):
    print('Index:',i)
    x_orig = [dataset.val_X[test_idx[i]], dataset.val_Y[test_idx[i]]]
    x_orig_no_max = [dataset.val_X_nomax[test_idx[i]], dataset.val_Y_nomax[test_idx[i]]]
    orig_label = np.argmax(dataset.val_Z[test_idx[i]])
    orig_preds=  model.predict(x_orig[0], x_orig[1])
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if np.argmax(orig_preds) != orig_label:
        print('skipping wrong classifed ..')
        print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig[1]))
    '''
    if x_len >= 30:
        print('skipping too long input..')
        print('--------------------------')
        continue
    '''
    # if np.max(orig_preds) < 0.90:
    #    print('skipping low confidence .. \n-----\n')
    #    continue
    
    target_list = [0,1,2]
    target_list.remove(orig_label)
    for target_label in target_list:
        print('****** ', len(test_list) + 1, ' ********')
        test_list.append(test_idx[i])
        orig_list.append(x_orig)
        orig_list_no_max.append(x_orig_no_max)
        #target_label = 1 if orig_label == 2 else 2
        orig_label_list.append(orig_label)
        x_adv = ga_atttack.attack( x_orig, target_label)
        adv_list.append(x_adv)
        if x_adv is None:
            print('%d failed' %(i+1))
            dist_list.append(100000)
        else:
            num_changes = np.sum(x_orig[1] != x_adv)
            print('%d - %d changed.' %(i+1, num_changes))
            dist_list.append(num_changes)
            # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)
        print('--------------------------')
        #if (len(test_list)>= 50):
        #    break

Shortest sentence in our test set is 30 words
Index: 0
skipping wrong classifed ..
--------------------------
Index: 1
skipping wrong classifed ..
--------------------------
Index: 2
skipping wrong classifed ..
--------------------------
Index: 3
skipping wrong classifed ..
--------------------------
Index: 4
******  1  ********
		 0  --  0.40092692
		 1  --  0.47402415
5 - 1 changed.
--------------------------
******  2  ********


/home/haejuu/aaai2019/haejuu/lib/python3.4/site-packages/ipykernel_launcher.py:54: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


		 0  --  0.33548075
		 1  --  0.42016214
5 - 1 changed.
--------------------------
Index: 5
skipping wrong classifed ..
--------------------------
Index: 6
******  3  ********
		 0  --  0.051025942
		 1  --  0.2264267
		 2  --  0.30189252
		 3  --  0.4647529
		 4  --  0.5563144
7 - 1 changed.
--------------------------
******  4  ********
		 0  --  0.01855124
		 1  --  0.114131585
		 2  --  0.17512561
		 3  --  0.2103832
		 4  --  0.24902499
		 5  --  0.31039798
		 6  --  0.31039798
		 7  --  0.31039798
		 8  --  0.31039798
		 9  --  0.31039798
		 10  --  0.31039798
		 11  --  0.31039798
		 12  --  0.31039798
		 13  --  0.31039798
		 14  --  0.31039798
		 15  --  0.31039798
		 16  --  0.31039798
		 17  --  0.31039798
		 18  --  0.31039798
		 19  --  0.31039798
		 20  --  0.31039798
		 21  --  0.31039798
		 22  --  0.31039798
		 23  --  0.31039798
		 24  --  0.31039798
		 25  --  0.31039798
		 26  --  0.31039798
		 27  --  0.31039798
		 28  --  0.31039798
		 29  --  0.31039798
7 failed

		 19  --  0.124292605
		 20  --  0.124292605
		 21  --  0.124292605
		 22  --  0.124292605
		 23  --  0.124292605
		 24  --  0.124292605
		 25  --  0.124292605
		 26  --  0.124292605
		 27  --  0.124292605
		 28  --  0.124292605
		 29  --  0.124292605
31 failed
--------------------------
Index: 31
******  41  ********
		 0  --  0.41226655
		 1  --  0.6116308
32 - 1 changed.
--------------------------
******  42  ********
		 0  --  0.2987586
		 1  --  0.7506255
32 - 1 changed.
--------------------------
Index: 32
******  43  ********
		 0  --  0.25835398
		 1  --  0.43589386
33 - 1 changed.
--------------------------
******  44  ********
		 0  --  0.41718203
33 - 1 changed.
--------------------------
Index: 33
skipping wrong classifed ..
--------------------------
Index: 34
******  45  ********
		 0  --  0.11650925
		 1  --  0.13669433
		 2  --  0.18171927
		 3  --  0.18171927
		 4  --  0.19648936
		 5  --  0.19648936
		 6  --  0.19753222
		 7  --  0.45100552
35 - 1 changed.
----------

		 23  --  0.25551373
		 24  --  0.25551373
		 25  --  0.25551373
		 26  --  0.25551373
		 27  --  0.25551373
		 28  --  0.25551373
		 29  --  0.25551373
59 failed
--------------------------
******  70  ********
		 0  --  0.21904036
		 1  --  0.24312499
		 2  --  0.25659013
		 3  --  0.25659013
		 4  --  0.2565901
		 5  --  0.2565901
		 6  --  0.2565901
		 7  --  0.25659013
		 8  --  0.25659013
		 9  --  0.25659013
		 10  --  0.25659013
		 11  --  0.25659013
		 12  --  0.25659013
		 13  --  0.25659013
		 14  --  0.25659013
		 15  --  0.25659013
		 16  --  0.25659013
		 17  --  0.25659013
		 18  --  0.25659013
		 19  --  0.25659013
		 20  --  0.25659013
		 21  --  0.25659013
		 22  --  0.25659013
		 23  --  0.25659013
		 24  --  0.25659013
		 25  --  0.25659013
		 26  --  0.25659013
		 27  --  0.25659013
		 28  --  0.25659013
		 29  --  0.25659013
59 failed
--------------------------
Index: 59
******  71  ********
		 0  --  0.12751518
		 1  --  0.50032884
60 - 1 changed.
---------------

...

		 0  --  0.43168518
1278 - 1 changed.
--------------------------
Index: 1278
skipping wrong classifed ..
--------------------------
Index: 1279
******  1601  ********
		 0  --  0.44961852
1280 - 1 changed.
--------------------------
******  1602  ********
		 0  --  0.4083751
1280 - 1 changed.
--------------------------
Index: 1280
skipping wrong classifed ..
--------------------------
Index: 1281
skipping wrong classifed ..
--------------------------
Index: 1282
skipping wrong classifed ..
--------------------------
Index: 1283
skipping wrong classifed ..
--------------------------
Index: 1284
skipping wrong classifed ..
--------------------------
Index: 1285
skipping wrong classifed ..
--------------------------
Index: 1286
skipping wrong classifed ..
--------------------------
Index: 1287
******  1603  ********
		 0  --  0.36055523
		 1  --  0.57978946
1288 - 1 changed.
--------------------------
******  1604  ********
		 0  --  0.36302796
1288 - 1 changed.
-------------------------

		 0  --  0.008971776
		 1  --  0.014732452
		 2  --  0.02176354
		 3  --  0.02176354
		 4  --  0.025888622
		 5  --  0.025888622
		 6  --  0.025888622
		 7  --  0.025888622
		 8  --  0.025888622
		 9  --  0.025888622
		 10  --  0.025888622
		 11  --  0.025888622
		 12  --  0.025888622
		 13  --  0.025888622
		 14  --  0.025888622
		 15  --  0.025888622
		 16  --  0.025888622
		 17  --  0.025888622
		 18  --  0.025888622
		 19  --  0.025888622
		 20  --  0.025888622
		 21  --  0.025888622
		 22  --  0.025888622
		 23  --  0.025888622
		 24  --  0.025888622
		 25  --  0.025888622
		 26  --  0.025888622
		 27  --  0.025888622
		 28  --  0.025888622
		 29  --  0.025888622
1320 failed
--------------------------
******  1642  ********
		 0  --  0.04952612
		 1  --  0.064670615
		 2  --  0.07617157
		 3  --  0.07732292
		 4  --  0.09156012
		 5  --  0.09156012
		 6  --  0.09170972
		 7  --  0.09170972
		 8  --  0.09170972
		 9  --  0.09618176
		 10  --  0.09618176
		 11  --  0.09618176
		 12

		 0  --  0.44625458
1353 - 1 changed.
--------------------------
******  1670  ********
		 0  --  0.36553252
1353 - 1 changed.
--------------------------
Index: 1353
******  1671  ********
		 0  --  0.3379678
		 1  --  0.4831543
1354 - 1 changed.
--------------------------
******  1672  ********
		 0  --  0.16100386
		 1  --  0.38307938
1354 - 1 changed.
--------------------------
Index: 1354
******  1673  ********
		 0  --  0.21093448
		 1  --  0.24982479
		 2  --  0.34894064
		 3  --  0.3639941
1355 - 1 changed.
--------------------------
******  1674  ********
		 0  --  0.28618228
		 1  --  0.3588105
		 2  --  0.40630448
1355 - 1 changed.
--------------------------
Index: 1355
skipping wrong classifed ..
--------------------------
Index: 1356
******  1675  ********
		 0  --  0.5098646
1357 - 1 changed.
--------------------------
******  1676  ********
		 0  --  0.24847277
		 1  --  0.38657758
1357 - 1 changed.
--------------------------
Index: 1357
******  1677  ********
		 0  --  

		 0  --  0.53006965
1389 - 1 changed.
--------------------------
Index: 1389
******  1719  ********
		 0  --  0.41874817
1390 - 1 changed.
--------------------------
******  1720  ********
		 0  --  0.05660703
		 1  --  0.33809084
		 2  --  0.3939935
1390 - 1 changed.
--------------------------
Index: 1390
skipping wrong classifed ..
--------------------------
Index: 1391
skipping wrong classifed ..
--------------------------
Index: 1392
******  1721  ********
		 0  --  0.26044366
		 1  --  0.28994313
		 2  --  0.293993
		 3  --  0.31894374
		 4  --  0.32440373
		 5  --  0.34393647
		 6  --  0.34605065
		 7  --  0.3529352
		 8  --  0.3529352
		 9  --  0.41569135
1393 - 1 changed.
--------------------------
******  1722  ********
		 0  --  0.4034283
		 1  --  0.43674216
1393 - 1 changed.
--------------------------
Index: 1393
skipping wrong classifed ..
--------------------------
Index: 1394
******  1723  ********
		 0  --  0.41296247
		 1  --  0.5474802
1395 - 1 changed.
-------------

ValueError: Error when checking : expected input_2 to have shape (None, 30) but got array with shape (0, 1)

### Visualize some output results

In [32]:
print('Median number of modifications ' , np.median([x for x in dist_list if x != 100000]))

Median number of modifications  1.0


In [33]:
orig_len = [np.sum(np.sign(x)) for x in orig_list[1]]
ratio = [dist_list[i]/orig_len[i] for i in range(len(orig_list[1]))]

In [34]:
ratio_success = [x for x in ratio if x < 1.0]
np.mean(ratio_success)

0.04444444444444444

In [ ]:
#visual_idx = 0
for visual_idx in range(len(adv_list)):
    print(visual_idx+1)
    display_utils.visualize_attack(model, dataset, orig_list_no_max[visual_idx], adv_list[visual_idx])

1
Original Prediction = entailment. (Confidence = 52.13) 
Premise


Hypothesis


---------  After attack -------------
New Prediction = neutral. (Confidence = 47.98) 


2
Original Prediction = entailment. (Confidence = 52.13) 
Premise


Hypothesis


...

Hypothesis


---------  After attack -------------
New Prediction = neutral. (Confidence = 47.41) 


452
Original Prediction = contradiction. (Confidence = 59.35) 
Premise


Hypothesis


---------  After attack -------------
New Prediction = entailment. (Confidence = 44.25) 
